## Average Citizens. ETH Analysis

In [ ]:
import matplotlib.pyplot as plt
import matplotlib as mpl
from IPython.display import HTML, display

from config import PROJECT_ID, ETH_ANALYSIS_TYPE, ETH_ANALYSIS_DATASET_NAME, INITIAL_TS, SNAPSHOT_TS
from utils import drop_table, create_table, get_df

DROP_TABLES = True
CREATE_TABLES = True

### Distribution Rule
<table style="text-align: center">
    <thead>
        <tr>
            <th rowspan=2></th>
            <th colspan=3>Grade</th>
        </tr>
        <tr>
            <th>1</th>
            <th>2</th>
            <th>3</th>
        </tr>
    </thead>
    <tbody>
        <tr>
            <td style="text-align: left"> ETH balance > 0.01 ETH and Amount of outgoing tx</td>
            <td> > 3 tx </td>
            <td> > 21 tx </td>
            <td> > 100 tx </td>
        </tr>
        <tr>
            <td style="text-align: left">OR ETH balance</td>
            <td> > 1 ETH</td>
            <td> > 10 ETH</td>
            <td> > 100 ETH</td>
        </tr>
    </tbody>
</table>